# AES Image Encryption and Decryption
---
This Jupyter Notebook demonstrates an AES image encryption and decryption application using the `pycryptodome` and `Pillow` libraries.
The application allows the user to encrypt and decrypt image files with a specified password.

## Step 1: Installation
Ensure you have Python 3.x installed. Install the required dependencies using pip:
```bash
pip install pycryptodome pillow
```

## Step 2: Import Libraries
We need to import necessary libraries for encryption, decryption, and handling images.

In [1]:
from Crypto.Cipher import AES
from Crypto.Protocol.KDF import scrypt
from Crypto.Random import get_random_bytes
from PIL import Image
import os
import io
from IPython.display import display
from ipywidgets import widgets, interact

## Step 3: Define Encryption and Decryption Functions
These functions will handle the encryption and decryption of image files using AES.

### Encryption Function
The `encrypt_image` function encrypts an image file using AES in GCM mode:
- Generates a key and nonce from the password using `scrypt`
- Reads the input image
- Encrypts the image data
- Writes the salt, nonce, tag, and ciphertext to the output file

In [2]:
def encrypt_image(input_file, output_file, password):
    # Generate a key and nonce from the password using scrypt
    salt = get_random_bytes(16)
    key = scrypt(password, salt, 32, N=2**14, r=8, p=1)
    nonce = get_random_bytes(16)

    # Initialize AES cipher in GCM mode
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)

    # Read the input image
    with open(input_file, 'rb') as f:
        image_data = f.read()

    # Encrypt the image data
    ciphertext, tag = cipher.encrypt_and_digest(image_data)

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    # Write salt, nonce, tag, and ciphertext to the output file
    with open(output_file, 'wb') as f:
        f.write(salt)
        f.write(nonce)
        f.write(tag)
        f.write(ciphertext)

    print(f'Encryption successful. Encrypted image saved at: {output_file}')

### Decryption Function
The `decrypt_image` function decrypts an AES-encrypted image file:
- Reads the salt, nonce, tag, and ciphertext from the input file
- Generates a key from the password and salt using `scrypt`
- Initializes the AES cipher in GCM mode
- Decrypts the ciphertext
- Saves the decrypted image

In [3]:
def decrypt_image(input_file, output_file, password):
    try:
        # Read salt, nonce, tag, and ciphertext from the input file
        with open(input_file, 'rb') as f:
            salt = f.read(16)
            nonce = f.read(16)
            tag = f.read(16)
            ciphertext = f.read()

        # Generate key from password and salt using scrypt
        key = scrypt(password, salt, 32, N=2**14, r=8, p=1)

        # Initialize AES cipher in GCM mode
        cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)

        # Decrypt the ciphertext
        decrypted_data = cipher.decrypt_and_verify(ciphertext, tag)

        # Create an image from decrypted data
        decrypted_image = Image.open(io.BytesIO(decrypted_data))

        # Ensure the output directory exists
        os.makedirs(os.path.dirname(output_file), exist_ok=True)

        # Save the decrypted image with a valid image extension
        decrypted_image.save(output_file)

        print(f'Decryption successful. Decrypted image saved at: {output_file}')

    except ValueError:
        print("Decryption failed. The provided key is incorrect. Please try again with the correct key.")

## Step 4: Interactive User Input
These functions will handle user input for choosing the operation (encryption or decryption) and selecting the image file.

In [4]:
def get_operation_choice():
    operation = widgets.ToggleButtons(
        options=['Encrypt', 'Decrypt'],
        description='Operation:',
        disabled=False,
        button_style='',
        tooltips=['Encrypt an image', 'Decrypt an image']
    )
    display(operation)
    return operation

def get_image_path():
    uploader = widgets.FileUpload(
        accept='.png, .jpg, .jpeg, .bmp, .gif',
        multiple=False
    )
    display(uploader)
    return uploader

## Step 5: Main Application Logic
This part of the notebook integrates the above functions to perform the encryption or decryption based on user input.

In [5]:
operation_widget = get_operation_choice()
image_uploader = get_image_path()

def on_button_clicked(b):
    operation = operation_widget.value
    image_file = next(iter(image_uploader.value))  # Get the uploaded image file
    image_content = image_uploader.value[image_file]['content']
    image_path = f'./{image_file}'
    with open(image_path, 'wb') as f:
        f.write(image_content)

    # Determine output file paths
    file_directory = os.path.dirname(image_path)
    image_filename = os.path.basename(image_path)
    image_name, image_extension = os.path.splitext(image_filename)

    if operation == 'Encrypt':
        output_file = os.path.join(file_directory, image_name + '_encrypted.enc')
    elif operation == 'Decrypt':
        output_file = os.path.join(file_directory, image_name + '_decrypted.png')

    # Get password
    password = input("Enter password (must be at least 16 characters long): ").encode('utf-8')  # Convert to bytes

    # Perform chosen operation
    if operation == 'Encrypt':
        encrypt_image(image_path, output_file, password)
    elif operation == 'Decrypt':
        decrypt_image(image_path, output_file, password)

button = widgets.Button(description="Run")
button.on_click(on_button_clicked)
display(button)

## Usage Instructions
1. Clone this repository from GitHub:
```bash
git clone https://github.com/muhammad-tarrab/Tarrab-Cybersecurity-Portfolio.git:
```
2. Navigate to the repository directory and install the required dependencies:
```bash
pip install pycryptodome pillow
```
3. Open the Jupyter Notebook:
```bash
jupyter notebook aes_image_encryption.ipynb
```
4. Follow the instructions in the notebook to encrypt and decrypt image files.

## License
This project is licensed under the MIT License - see the [LICENSE](LICENSE) file for details.